## GROMACS Tutorial
### http://www.mdtutorials.com/gmx/complex/01_pdb2gmx.html

### Step 1-1: Prepare the Protein Topology

In [ ]:
srun --cpus-per-task=8 --mem=8GB --time=03:00:00 --pty /bin/bash

In [ ]:
# downloaded the structure
wget http://www.mdtutorials.com/gmx/complex/Files/3HTB_clean.pdb

In [ ]:
# save the protein and JZ4 ligand into separate coordinate files
grep JZ4 3HTB_clean.pdb > jz4.pdb

In [ ]:
# Manually delete the JZ4 lines from 3HTB_clean.pdb

# HETATM 1376  C4  JZ4 A 167      24.294 -24.124  -0.071  1.00 16.39           C  
# HETATM 1377  C7  JZ4 A 167      21.553 -27.214  -4.112  1.00 16.92           C  
# HETATM 1378  C8  JZ4 A 167      22.068 -26.747  -5.331  1.00 14.84           C  
# HETATM 1379  C9  JZ4 A 167      22.671 -25.512  -5.448  1.00 15.93           C  
# HETATM 1380  C10 JZ4 A 167      22.769 -24.730  -4.295  1.00 15.43           C  
# HETATM 1381  C11 JZ4 A 167      21.693 -26.459  -2.954  1.00 16.72           C  
# HETATM 1382  C12 JZ4 A 167      22.294 -25.187  -3.075  1.00 14.13           C  
# HETATM 1383  C13 JZ4 A 167      22.463 -24.414  -1.808  1.00 15.94           C  
# HETATM 1384  C14 JZ4 A 167      23.925 -24.704  -1.394  1.00 15.28           C  
# HETATM 1385  OAB JZ4 A 167      23.412 -23.536  -4.342  1.00 13.88           O  

In [ ]:
# download the latest CHARMM36 force field tarball and the "cgenff_charmm2gmx.py" conversion script
wget https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-jul2022.ff.tgz
wget https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/cgenff_charmm2gmx_py3_nx2.py

# Unarchive the force field tarball
tar -zxvf charmm36-jul2022.ff.tgz

In [ ]:
module load gromacs/openmpi/intel/2020.4

In [ ]:
gmx_mpi pdb2gmx -f 3HTB_clean.pdb -o 3HTB_processed.gro -ter

# Select the Force Field:                1: CHARMM all-atom force field
# Select the Water Model:                1: TIP3P      CHARMM-modified TIP3P water model (recommended over original TIP3P)
# Select start terminus type for MET-1:  1: NH3+
# Start terminus MET-1: NH3+             0: COO-

### Step 1-2 : Prepare the Ligand Topology

In [ ]:
# Download Avogadro program
# Open jz4.pdb in Avogadro
# from the "Build" menu, choose "Add Hydrogens." Avogadro will build all of the H atoms onto the JZ4 ligand. 
# Save a .mol2 file (File -> Save As... and choose Sybyl Mol2 from the drop-down menu) named "jz4.mol2."

# open jz4.mol2 in plain-text editor
# fix the residue names and numbers such that they are all the same
# save

In [ ]:
# fix strange bond order
wget http://www.mdtutorials.com/gmx/complex/Files/sort_mol2_bonds.pl
perl sort_mol2_bonds.pl jz4.mol2 jz4_fix.mol2

In [ ]:
"""
A poor ligand topology can lead to significant wasted time and unreliable results. 
# Always validate the topologies of newly parametrized species! 
# At minimum, check the magnitudes of charges and the atom types assigned to the ligand against existing moieties in the force field. 
"""

# Generate the JZ4 Topology with CGenFF
# Visit the CGenFF server, 
# log into your account
# click "Upload molecule" at the top of the page. 
# Upload jz4_fix.mol2 and 
# Save its contents from your web browser into a file called "jz4.str." 

# For consistency, download from the tutorial
wget http://www.mdtutorials.com/gmx/complex/Files/jz4.str

# The CHARMM stream file contains all of the topology information - atom types, charges, and bonded connectivity. 
# CGenFF also provides penalty scores for each parameter, that is, an assessment of how reliable the assigned parameter is. 
# Anything below 10 is considered acceptable for immediate use. 
# Values from 10 - 50 imply that some validation of the topology is warranted, 
# Any penalties larger than 50 generally require manual reparametrization. 
# This penalty scoring is one of the most important features of the CGenFF server.

In [ ]:
# version match with cgenff_charmm2gmx_py3_nx2.py
pip install networkx==2.3
python cgenff_charmm2gmx_py3_nx2.py JZ4 jz4_fix.mol2 jz4.str charmm36-jul2022.ff

# This ligand introduces new bonded parameters and these parameters are written to a file called "jz4.prm," which is in the format of a GROMACS .itp file. 
# The ligand topology is now written to "jz4.itp," which contains the ligand [ moleculetype ] definition.

### Step 2: Build the Complex

In [ ]:
gmx_mpi editconf -f jz4_ini.pdb -o jz4.gro

In [ ]:
# Copy 3HTB_processed.gro to a new file to be manipulated, call it "complex.gro," 
# Next, simply copy the coordinate section of jz4.gro and paste it into complex.gro
# No space line in complex.gro between protein and ligand (atom 2615)

In [ ]:
# Build the Topology

# In topol.top file add
"""
; Include ligand topology
#include "jz4.itp
""""

"""
; Include ligand parameters
#include "jz4.prm"
"""

"""
JZ4                 1
"""

### Step 3 - Define Box and Solvate

In [1]:

gmx_mpi editconf -f complex.gro -o newbox.gro -bt dodecahedron -d 1.0
gmx_mpi solvate -cp newbox.gro -cs spc216.gro -p topol.top -o solv.gro

SyntaxError: invalid syntax (1709253273.py, line 1)

### Step 4 - Add Ions

In [ ]:
# Use grompp to assemble a .tpr file, using any .mdp file.
wget http://www.mdtutorials.com/gmx/complex/Files/ions.mdp

gmx_mpi grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 100
# gmx_mpi grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr
# Warning: atom name 2613 in topol.top and solv.gro does not match (OT1 - O1)
# Warning: atom name 2614 in topol.top and solv.gro does not match (OT2 - O2)
# WARNING 1 [file topol.top, line 24632]:
#   2 non-matching atom names
#   atom names from topol.top will be used
#   atom names from solv.gro will be ignored


# pass our .tpr file to genion:
gmx_mpi genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutral
# Select: Group 15 (SOL) has 30882 elements
# In topol.top file 
# [ molecules ]
# ; Compound        #mols
# Protein_chain_A     1
# JZ4                 1
# SOL             10228
# CL                  6

### Step 5 - Energy Minimization

In [ ]:
wget http://www.mdtutorials.com/gmx/complex/Files/em.mdp

gmx_mpi grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr

gmx_mpi mdrun -v -deffnm em
# Steepest Descents converged to Fmax < 1000 in 151 steps
# Potential Energy  = -4.9001400e+05
# Maximum force     =  9.7044019e+02 on atom 2045
# Norm of force     =  5.9329782e+01

### Step 6 - Equilibration

In [ ]:
# Applying restraints to the ligand
# Generate a position restraint topology

# First, create an index group for JZ4 that contains only its non-hydrogen atoms:
gmx_mpi make_ndx -f jz4.gro -o index_jz4.ndx
 > 0 & ! a H*
 > q

In [ ]:
# Then, execute the genrestr module and select this newly created index group
gmx_mpi genrestr -f jz4.gro -n index_jz4.ndx -o posre_jz4.itp -fc 1000 1000 1000
# Selected 3: 'System_&_!H*'

In [ ]:
# Include information in the topol.top file

"""
; Ligand position restraints
#ifdef POSRES
#include "posre_jz4.itp"
#endif
"""

In [ ]:
# Treatment of temperature coupling groups
# merges the protein and JZ4
gmx_mpi make_ndx -f em.gro -o index.ndx
> 1 | 13
> q

In [ ]:
# NVT equilibration
wget http://www.mdtutorials.com/gmx/complex/Files/nvt.mdp
gmx_mpi grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -n index.ndx -o nvt.tpr
gmx_mpi mdrun -deffnm nvt

# ls -l
# -rw-rw-r--+ 1 qo210 qo210   805800 Dec 16 00:23  nvt.cpt
# -rw-rw-r--+ 1 qo210 qo210    57656 Dec 16 00:23  nvt.edr
# -rw-rw-r--+ 1 qo210 qo210  2312029 Dec 16 00:23  nvt.gro
# -rw-rw-r--+ 1 qo210 qo210    84839 Dec 16 00:23  nvt.log

In [ ]:
# NPT equilibration
wget http://www.mdtutorials.com/gmx/complex/Files/npt.mdp
gmx_mpi grompp -f npt.mdp -c nvt.gro -t nvt.cpt -r nvt.gro -p topol.top -n index.ndx -o npt.tpr
gmx_mpi mdrun -deffnm npt

### Step 7 - Production MD

In [ ]:
gmx_mpi grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -n index.ndx -o md_0_10.tpr
gmx_mpi mdrun -deffnm md_0_10

### Step 8 - Analysis - Analyzing Protein-Ligand Interactions and Ligand Dynamics

In [ ]:
# Recentering and Rewrapping Coordinates
gmx_mpi trjconv -s md_0_10.tpr -f md_0_10.xtc -o md_0_10_center.xtc -center -pbc mol -ur compact
# Selected 1: 'Protein'
# Selected 0: 'System'

# extract the first frame (t = 0 ns) of the trajectory
gmx_mpi trjconv -s md_0_10.tpr -f md_0_10_center.xtc -o start.pdb -dump 0
# Selected 0: 'System'

# perform rotational and translational fitting
gmx_mpi trjconv -s md_0_10.tpr -f md_0_10_center.xtc -o md_0_10_fit.xtc -fit rot+trans
# Selected 4: 'Backbone'                                                 
# Selected 0: 'System'

In [ ]:
# Analyzing Protein-Ligand Interactions and Ligand Dynamics
gmx_mpi distance -s md_0_10.tpr -f md_0_10_center.xtc -select 'resname "JZ4" and name OAB plus resid 102 and name OE1' -oall

#determining the presence of a hydrogen bond is the angle formed among the donor, hydrogen, and acceptor atoms
gmx_mpi make_ndx -f em.gro -o index.ndx
...
> 13 & a OAB | a H12
# Creates group 22(should be 23)
# Copied index group 13 'JZ4'
# Found 1 atoms with name OAB
# Merged two groups with AND: 22 1 -> 1
# Found 1 atoms with name H12
# Merged two groups with OR: 1 1 -> 2
# 22 JZ4_&_OAB_H12       :     2 atoms  (creates group 23)


> 1 & r 102 & a OE1
# Creates group 23 (should be 24)
# Copied index group 1 'Protein'
# Merged two groups with AND: 2614 17 -> 17
# Found 14 atoms with name OE1
# Merged two groups with AND: 17 14 -> 1
# 23 Protein_&_r_102_&_OE1:     1 atoms

# > 23 | 24
> 22 | 23
# Copied index group 22 'JZ4_&_OAB_H12'
# Copied index group 23 'Protein_&_r_102_&_OE1'
# Merged two groups with OR: 2 1 -> 3
#  25 JZ4_&_OAB_H12_Protein_&_r_102_&_OE1:     3 atoms

> q

In [ ]:
# Analyze the angle formed by these three atoms using the angle module
gmx_mpi angle -f md_0_10_center.xtc -n index.ndx -ov angle.xvg

# Select a group: 25
# Selected 25: 'JZ4_&_OAB_H12_Protein_&_r_102_&_OE1'
# Last frame         15 time  150.000
# Found points in the range from 20 to 113 (max 180)
#  < angle >  = 42.1667

# The outcome of the calculation is the acceptor-donor-hydrogen angle
# To get the desired angle of donor-hydrogen-acceptor, 
# we would have to manually edit the index.ndx file in a text file to reorder the atom numbers 
# In the index.ndx file
# [ JZ4_&_OAB_H12_Protein_&_r_102_&_OE1 ]
# 1616 2624 2636
# change to 2624 2636 1616

In [ ]:
# Re-analyze the angle formed by these three atoms using the angle module
gmx_mpi angle -f md_0_10_center.xtc -n index.ndx -ov angle.xvg

# Select a group: 25
# Selected 25: 'JZ4_&_OAB_H12_Protein_&_r_102_&_OE1'
# Last frame         15 time  150.000

# Back Off! I just backed up angle.xvg to ./#angle.xvg.1#
# Found points in the range from 60 to 154 (max 180)
#  < angle >  = 130.982

In [ ]:
# To compute a heavy-atom RMSD of just JZ4, create a new index group for it:

gmx_mpi make_ndx -f em.gro -n index.ndx

> 13 & ! a H*
# Copied index group 13 'JZ4'
# Found 21901 atoms with name H*
# Complemented group: 11605 atoms
# Merged two groups with AND: 22 11605 -> 10

#  26 JZ4_&_!H*           :    10 atoms

> name 26 JZ4_Heavy

> q

In [ ]:
gmx_mpi rms -s em.tpr -f md_0_10_center.xtc -n index.ndx -tu ns -o rmsd_jz4.xvg

# Selected 4: 'Backbone'
# Selected 26: 'JZ4_Heavy'

### Step 8 - Analysis - Protein-Ligand Interaction Energy

In [ ]:
wget http://www.mdtutorials.com/gmx/complex/Files/ie.mdp

In [ ]:
gmx_mpi grompp -f ie.mdp -c npt.gro -t npt.cpt -p topol.top -n index.ndx -o ie.tpr

In [ ]:
gmx_mpi mdrun -deffnm ie -rerun md_0_10.xtc -nb cpu

In [2]:
!pwd

/scratch/work/courses/CHEM-GA-2671-2023fa/students/qo210/comp-lab-class-2023/FinalProject_new
